In [72]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings

# 1. Data Preparation

In [3]:
trainDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\df_train_raMSIn4nonInDI_STDnorm.csv")
diDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\df_nonInDI_raMSIn4nonInDI_STDnorm.csv")
cDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\df_nonInDI_raMSIn4nonInClinicsMSI3_STDnorm.csv")

In [4]:
trainDEFSDf

,pixel_id,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,280.2361,241.2173,311.2228,339.1996,353.2005,325.1842,250.1449,514.2846,265.1478,type
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.688111,0.572632,-1.174275,-0.402685,-0.452294,4.484746,-0.685990,0.523277,-0.553016,1.064326,-0.046503,2.433592,2.536437,3.236938,1.379780,-0.424850,4.516228,0
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,5.100711,1.502384,-0.377359,-0.402685,-0.245971,5.600166,-1.129480,0.254736,-0.685862,1.372705,-0.046503,3.295405,3.186475,4.400280,3.286631,-0.424850,5.492250,0
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.446238,0.633067,-0.825036,-0.402685,-0.436322,3.698358,-1.062778,-0.138344,-0.590147,0.989152,-0.046503,2.422201,2.616717,2.951801,2.283747,-0.424850,4.409029,0
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.197065,0.438803,-0.418424,-0.402685,-0.338349,3.487575,-1.009749,-0.087386,-0.714693,1.281385,-0.046503,2.330914,2.116008,2.725915,2.047144,-0.424850,4.249139,0
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,1.997365,0.762296,-1.300452,-0.402685,-0.327447,2.554686,-1.030780,0.621524,-0.431587,0.783422,-0.046503,1.465060,1.970677,1.668743,1.036566,-0.424850,2.983302,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.410648,-1.036221,0.041017,-0.402685,-0.904559,0.729069,-1.407607,1.081844,-1.162534,-1.168280,-0.046503,0.451770,0.671118,0.388205,0.417243,2.170259,0.504114,1
90956,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.252523,-1.036221,-0.899514,-0.402685,-0.389559,0.861528,-1.347767,1.340689,-1.305998,0.717326,-0.046503,0.530942,0.710665,0.407962,2.303768,-0.424850,0.259900,1
90957,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.357586,1.266093,-0.920759,-0.402685,-0.574288,0.514129,-0.809247,0.871351,-1.048595,-1.168280,-0.046503,0.493749,1.044021,0.412257,1.567262,-0.424850,-0.735083,1
90958,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.526709,-1.036221,-1.209449,-0.402685,-1.037316,0.694260,-0.977460,0.664582,-1.091385,-1.168280,-0.046503,0.357653,-0.691736,0.401013,2.067573,-0.424850,0.671797,1


In [5]:
can = []
for i in list(range(0,9)) + list(range(15,17)) + [len(list(trainDEFSDf.columns))-1]:
    can.append(list(trainDEFSDf.columns)[i])

dican = []
for i in list(range(0,9)) + list(range(15,17)) + [len(list(diDEFSDf.columns))-1]:
    dican.append(list(diDEFSDf.columns)[i])
    
ccan = []
for i in list(range(0,9)) + list(range(15,17)) + [len(list(cDEFSDf.columns))-1]:
    ccan.append(list(cDEFSDf.columns)[i])

In [6]:
can

['pixel_id',
 '311.1684',
 '269.2486',
 '215.0328',
 '295.2278',
 '883.5331',
 '309.1704',
 '738.5059',
 '435.2965',
 '250.1449',
 '514.2846',
 'type']

In [7]:
dican

['pixel_id',
 '311.169',
 '269.2486',
 '215.032',
 '295.2279',
 '883.5375',
 '309.1728',
 '738.5084',
 '435.2966',
 '250.145',
 '514.2844',
 'type']

In [8]:
ccan

['pixel_id',
 '311.1689',
 '269.2486',
 '215.0324',
 '295.2278',
 '883.5322',
 '309.1744',
 '738.5034',
 '435.2966',
 '250.1448',
 '448.3072',
 'type']

In [9]:
## input training set ## 90960 x 20 df
trainDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\df_train_raMSIn4nonInDI_STDnorm.csv")
#trainDEFSDf[trainDEFSDf.type .== 1, :]
## calculate weight ## 0: 47449, 1: 43511
trainDEFSDf = trainDEFSDf[can]
Yy_train = trainDEFSDf["type"]  # 0.9585; 1.0453
sampleW = []
for w in Yy_train:
    if w == 0:
        sampleW.append(0.9585)
    elif w == 1:
        sampleW.append(1.0453) 


In [10]:
## input ext val set ## 6075 x 20 df
extDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\df_ext_raMSIn4nonInDI_STDnorm.csv")
#extDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 2943, 1: 3132
extDEFSDf = extDEFSDf[can]
Yy_ext = extDEFSDf["type"]  # 1.0321; 0.9698
sampleExtW = []
for w in Yy_ext:
    if w == 0:
        sampleExtW.append(1.0321)
    elif w == 1:
        sampleExtW.append(0.9698) 


In [11]:
## input ingested set ## 97035 x 20 df
ingestedDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\df_ingested_raMSIn4nonInDI_STDnorm.csv")
#ingestedDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 50392, 1: 46643
ingestedDEFSDf = ingestedDEFSDf[can]
Yy_ingested = ingestedDEFSDf["type"]  # 0.9628; 1.0402
sampleIngestedW = []
for w in Yy_ingested:
    if w == 0:
        sampleIngestedW.append(0.9628)
    elif w == 1:
        sampleIngestedW.append(1.0402)


In [12]:
## input FNA set ## 88701 x 20 df
fnaDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\df_FNA_raMSIn4nonInDI_STDnorm.csv")
#fnaDEFSDf[fnaDEFSDf.type .== 1, :]
## calculate weight ##  0: 44540, 1: 44161
fnaDEFSDf = fnaDEFSDf[can]
Yy_FNA = fnaDEFSDf["type"]  # 0.9957; 1.0043
sampleFNAW = []
for w in Yy_FNA:
    if w == 0:
        sampleFNAW.append(0.9957)
    elif w == 1:
        sampleFNAW.append(1.0043)


In [13]:
## input DirectIn set ## 3028 x 20 df
diDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\df_nonInDI_raMSIn4nonInDI_STDnorm.csv")
#diDEFSDf[diDEFSDf.type .== 1, :]
## calculate weight ## 0: 1513, 1: 1515
diDEFSDf = diDEFSDf[dican]
diDEFSDf = diDEFSDf.rename(columns={"311.169": "311.1684", "215.032": "215.0328", "295.2279":"295.2278", "883.5375":"883.5331", 
                         "309.1728":"309.1704", "738.5084":"738.5059", "435.2966":"435.2965", "250.145":"250.1449", 
                         "514.2844":"514.2846"})
Yy_DI = diDEFSDf["type"]  # 1.0007; 0.9993
sampleDiW = []
for w in Yy_DI:
    if w == 0:
        sampleDiW.append(1.0007)
    elif w == 1:
        sampleDiW.append(0.9993)


In [14]:
## input Clinics set ## 6711 x 20 df
cDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\df_nonInDI_raMSIn4nonInClinicsMSI3_STDnorm.csv")
#diDEFSDf[diDEFSDf.type .== 1, :]
## calculate weight ## 0: 3449; 1: 3262
cDEFSDf = cDEFSDf[ccan]
cDEFSDf = cDEFSDf.rename(columns={"311.1689": "311.1684", "215.0324": "215.0324", "883.5322":"883.5331", 
                         "309.1744":"309.1704", "738.5034":"738.5059", "435.2966":"435.2965", "250.1448":"250.1449", 
                         "498.2896":"514.2846"})
Yy_c = cDEFSDf["type"]  # 0.9729; 1.0287
sampleCW = []
for w in Yy_c:
    if w == 0:
        sampleCW.append(0.9729)
    elif w == 1:
        sampleCW.append(1.0287)


In [15]:
diDEFSDf

,pixel_id,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.402342,0.913244,0.833746,-0.387446,0.488453,1.697873,0.601978,0.518836,2.971000,0.532073,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.314493,1.539995,0.003972,0.129889,0.246011,1.609584,0.390074,0.742545,2.805791,0.443193,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.841655,1.554217,0.670368,0.339209,0.950243,2.182584,1.755656,0.796962,3.286110,0.989125,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.621964,2.090087,0.272553,0.593092,1.079633,1.903016,0.849887,0.596528,3.016573,0.801811,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.303707,1.234845,0.156936,0.247718,0.548905,1.677419,0.944668,0.913738,3.116293,0.783867,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3023,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,0.046377,0.004282,1.384092,0.640055,1.053762,-0.172416,0.489131,0.529506,-0.437109,1.369131,1
3024,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-1.799346,-1.043795,-1.051923,-1.271685,0.599189,-1.114427,1.334575,-1.475923,0.064193,0.404445,1
3025,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.519249,-1.667011,1.280725,-0.144515,0.109578,-0.360029,0.180897,0.768313,-0.437109,1.725927,1
3026,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.419602,-0.812576,1.794612,-0.889368,0.995031,-0.357697,0.403818,-0.280928,-0.437109,1.301959,1


In [16]:
cDEFSDf

,pixel_id,311.1684,269.2486,215.0324,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,448.3072,type
0,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.180725,-0.876131,-0.053859,-0.672764,-1.247431,-0.561901,-1.461826,-0.434488,-0.118059,-0.600421,0
1,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.357559,-0.557249,0.071199,-0.672764,-0.885699,0.896613,-1.461826,-0.434488,0.141921,-0.600421,0
2,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.360496,-0.732288,0.443584,-0.672764,-1.247431,0.873763,-1.461826,-0.434488,0.228312,-0.600421,0
3,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.414136,-0.641708,0.276798,-0.672764,-1.247431,0.807055,-1.461826,-0.434488,0.188690,-0.600421,0
4,HKULiver_Post29wk_HCCAMCLiver_Clinics_Liver_S1...,0.291888,-0.550431,0.173477,-0.672764,-1.247431,0.742037,-1.461826,1.201794,0.099504,-0.600421,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6706,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_27_24,-0.533442,-1.203196,-0.483954,-0.672764,-1.247431,-0.561901,-1.461826,-0.434488,-1.551766,-0.600421,1
6707,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_33_25,-0.510905,-1.337545,-0.483954,-0.672764,-1.247431,-0.561901,-0.463838,-0.434488,-1.930156,-0.600421,1
6708,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_34_25,-0.272450,-1.091268,-0.483954,-0.672764,-1.247431,-0.561901,-0.498383,-0.434488,-1.733144,-0.600421,1
6709,HKULiver_Post29wk_HCCLiver_Clinics_Liver_S7_35_25,-0.445932,-1.194061,-0.483954,-0.672764,-1.247431,-0.561901,-0.549042,-0.434488,-1.408482,-0.600421,1


In [17]:
## define functions for performace evaluation ##

# Average score
def avgScore(arrAcc, cv):
    sumAcc = 0
    for acc in arrAcc:
        sumAcc += acc
    return sumAcc / cv


In [18]:
trainDEFSDf[trainDEFSDf.columns[1:-1]]

,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846
0,3.688111,0.572632,-1.174275,-0.402685,-0.452294,4.484746,-0.685990,0.523277,1.379780,-0.424850
1,5.100711,1.502384,-0.377359,-0.402685,-0.245971,5.600166,-1.129480,0.254736,3.286631,-0.424850
2,3.446238,0.633067,-0.825036,-0.402685,-0.436322,3.698358,-1.062778,-0.138344,2.283747,-0.424850
3,3.197065,0.438803,-0.418424,-0.402685,-0.338349,3.487575,-1.009749,-0.087386,2.047144,-0.424850
4,1.997365,0.762296,-1.300452,-0.402685,-0.327447,2.554686,-1.030780,0.621524,1.036566,-0.424850
...,...,...,...,...,...,...,...,...,...,...
90955,0.410648,-1.036221,0.041017,-0.402685,-0.904559,0.729069,-1.407607,1.081844,0.417243,2.170259
90956,0.252523,-1.036221,-0.899514,-0.402685,-0.389559,0.861528,-1.347767,1.340689,2.303768,-0.424850
90957,0.357586,1.266093,-0.920759,-0.402685,-0.574288,0.514129,-0.809247,0.871351,1.567262,-0.424850
90958,0.526709,-1.036221,-1.209449,-0.402685,-1.037316,0.694260,-0.977460,0.664582,2.067573,-0.424850


In [19]:
## define a function for Gradient Boost ##
def optimXGBoostClass(inputDB, inputDB_ingested, inputDB_ext, inputDB_FNA, inputDB_di):
    #lr_r = [15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5]  # 11
    #lr_r = [9.5, 10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5, 20.5]  # 12
    lr_r = [8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]  # 12
    #lr_r = [4.6, 4.62, 4.64, 4.66, 4.68, 4.7, 4.72, 4.74, 4.76, 4.78, 4.8, 4.82, 4.84, 4.86, 4.88, 4.9, 4.92, 4.94, 4.96, 4.98, 5, 5.02, 5.04, 5.06, 5.08, 5.1, 5.12, 5.14, 5.16, 5.18, 5.2, 5.22, 5.24, 5.26, 5.28, 5.3, 5.32, 5.34, 5.36, 5.38, 5.4, 5.42, 5.44, 5.46, 5.48, 5.5]  # 46
    #lr_r = [4.6, 4.61, 4.62, 4.63, 4.64, 4.65, 4.66, 4.67, 4.68, 4.69, 4.7, 4.71, 4.72, 4.73, 4.74, 4.75, 4.76, 4.77, 4.78, 4.79, 4.8]  # 21
    #cols_r = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]  # 7
    #cols_r = [0.8, 0.825, 0.85, 0.875, 0.9, 0.925, 0.95, 0.975, 1]  # 9
    cols_r = [0.65, 0.7, 0.75, 0.8, 0.85, 0.90, 0.95, 1]  # 8
    #cols_r = [1]  # 1
    #depth_r = [3, 4, 5, 6]  # 4
    depth_r = [3, 4, 5]  # 3
    #depth_r = [3, 4]  # 2
    #subs_r = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7]  # 7
    #subs_r = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]  # 9
    subs_r = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75]  # 6
    #tree_r = [100]  # 1
    #tree_r = [100, 150]  # 2
    tree_r = [50, 100]  # 2
    #tree_r = [200, 300]  # 2
    #tree_r = [25, 50, 75, 100, 200, 300, 500]  # 7
    #est_r = [8, 32] # 2
    #est_r = [5, 32, 64] # 3
    #est_r = [64, 96] # 2
    #est_r = [96, 128] # 2
    est_r = [2, 8, 16, 32, 64, 96] # 6

    rs = 42
    z = np.zeros(38)
    itr = 1

    N_train = inputDB
    M_train = inputDB_ingested
    M_ext = inputDB_ext
    M_FNA = inputDB_FNA
    M_di = inputDB_di

    for lr in lr_r:
        for c in cols_r:
            for d in depth_r:
                for s in subs_r:
                    for t in tree_r:
                        for est in est_r:
    
                            print("itr=", itr, ", lr=", lr, ", cols=", c, ", depth=", d, ", sub=", s, ", tree=", t, ", est=", est)
                            print("## loading in data ##")
                            Xx_train = M_train[M_train.columns[1:-1]]
                            nn_train = N_train[N_train.columns[1:-1]]
                            Xx_Ext = M_ext[M_ext.columns[1:-1]]
                            Xx_FNA = M_FNA[M_FNA.columns[1:-1]]
                            Xx_di = M_di[M_di.columns[1:-1]]
                            #
                            Yy_train = M_train["type"]
                            mm_train = N_train["type"]
                            Yy_Ext = M_ext["type"]
                            Yy_FNA = M_FNA["type"]
                            Yy_di = M_di["type"]
                            print("## Classification ##")
                            model_xgb = xgb.XGBClassifier(
                                base_score = 0.5, 
                                booster = 'gbtree', 
                                colsample_bylevel = 1, 
                                colsample_bynode = 1, 
                                colsample_bytree = c, 
                                enable_categorical = False, 
                                eval_metric = 'auc', 
                                gamma = 0, 
                                gpu_id = -1, 
                                grow_policy = 'lossguide', 
                                importance_type = None, 
                                learning_rate = lr, 
                                max_bin = 256, 
                                max_cat_to_onehot = 4, 
                                max_delta_step = 0, 
                                max_depth = d, 
                                max_leaves = 0, 
                                min_child_weight = 1, 
                                #missing = nan, 
                                monotone_constraints = '()', 
                                n_estimators = t, 
                                n_jobs = 0, 
                                num_parallel_tree = 1, 
                                objective = 'binary:hinge', 
                                predictor = 'auto', 
                                random_state = 0, 
                                reg_alpha = 0, 
                                reg_lambda = 1, 
                                sampling_method = 'uniform', 
                                scale_pos_weight = None, 
                                subsample = s, 
                                tree_method = 'hist', 
                                use_label_encoder = False, 
                                validate_parameters = 1, 
                                verbosity = None, 
                                early_stopping_rounds = est)

                            # Training Set
                            model_xgb.fit(Xx_train, Yy_train, 
                                          eval_set=[(Xx_FNA, Yy_FNA), (Xx_di, Yy_di)])

                            fit_params = {'eval_set': [[Xx_FNA, Yy_FNA], [Xx_di, Yy_di]]}

                            importances = permutation_importance(model_xgb, Xx_FNA, Yy_FNA, n_repeats=10, random_state=42, n_jobs=-1)
                            #print(importances["importances_mean"])

                            if itr == 1:
                                z[0] = lr
                                z[1] = c
                                z[2] = d
                                z[3] = s
                                z[4] = t
                                z[5] = est
                                z[6] = f1_score(mm_train, model_xgb.predict(nn_train), sample_weight=sampleW)
                                z[7] = matthews_corrcoef(mm_train, model_xgb.predict(nn_train), sample_weight=sampleW)
                                z[8] = f1_score(Yy_Ext, model_xgb.predict(Xx_Ext), sample_weight=sampleExtW)
                                z[9] = matthews_corrcoef(Yy_Ext, model_xgb.predict(Xx_Ext), sample_weight=sampleExtW)
                                print("## No CV ##")
                                scores_XGB = cross_val_score(model_xgb, Xx_train, Yy_train, cv=5, scoring = "f1", fit_params = fit_params)
                                z[10] = scores_XGB.mean()
                                #print(scores_XGB.mean())
                                z[11] = f1_score(Yy_FNA, model_xgb.predict(Xx_FNA), sample_weight=sampleFNAW)
                                z[12] = matthews_corrcoef(Yy_FNA, model_xgb.predict(Xx_FNA), sample_weight=sampleFNAW)
                                z[13] = recall_score(Yy_FNA, model_xgb.predict(Xx_FNA))
                                z[14] = f1_score(Yy_di, model_xgb.predict(Xx_di), sample_weight=sampleDiW)
                                z[15] = matthews_corrcoef(Yy_di, model_xgb.predict(Xx_di), sample_weight=sampleDiW)
                                z[16] = recall_score(Yy_di, model_xgb.predict(Xx_di))
                                z[17] = rs
                                z[18] = importances["importances_mean"][0]
                                z[19] = importances["importances_mean"][1]
                                z[20] = importances["importances_mean"][2]
                                z[21] = importances["importances_mean"][3]
                                z[22] = importances["importances_mean"][4]
                                z[23] = importances["importances_mean"][5]
                                z[24] = importances["importances_mean"][6]
                                z[25] = importances["importances_mean"][7]
                                z[26] = importances["importances_mean"][8]
                                z[27] = importances["importances_mean"][9]
                                z[28] = importances["importances_std"][0]
                                z[29] = importances["importances_std"][1]
                                z[30] = importances["importances_std"][2]
                                z[31] = importances["importances_std"][3]
                                z[32] = importances["importances_std"][4]
                                z[33] = importances["importances_std"][5]
                                z[34] = importances["importances_std"][6]
                                z[35] = importances["importances_std"][7]
                                z[36] = importances["importances_std"][8]
                                z[37] = importances["importances_std"][9]
                            else:
                                itrain = f1_score(mm_train, model_xgb.predict(nn_train), sample_weight=sampleW)
                                jtrain = matthews_corrcoef(mm_train, model_xgb.predict(nn_train), sample_weight=sampleW)
                                ival = f1_score(Yy_Ext, model_xgb.predict(Xx_Ext), sample_weight=sampleExtW)
                                jval = matthews_corrcoef(Yy_Ext, model_xgb.predict(Xx_Ext), sample_weight=sampleExtW)
                                print("## No CV ##")
                                scores_XGB = cross_val_score(model_xgb, Xx_train, Yy_train, cv=5, scoring = "f1", fit_params = fit_params)
                                traincvtrain = scores_XGB.mean()
                                f1s = f1_score(Yy_FNA, model_xgb.predict(Xx_FNA), sample_weight=sampleFNAW)
                                mccs = matthews_corrcoef(Yy_FNA, model_xgb.predict(Xx_FNA), sample_weight=sampleFNAW)
                                rec = recall_score(Yy_FNA, model_xgb.predict(Xx_FNA))
                                f1s2 = f1_score(Yy_di, model_xgb.predict(Xx_di), sample_weight=sampleDiW)
                                mccs2 = matthews_corrcoef(Yy_di, model_xgb.predict(Xx_di), sample_weight=sampleDiW)
                                rec2 = recall_score(Yy_di, model_xgb.predict(Xx_di))
                                im1 = importances["importances_mean"][0]
                                im2 = importances["importances_mean"][1]
                                im3 = importances["importances_mean"][2]
                                im4 = importances["importances_mean"][3]
                                im5 = importances["importances_mean"][4]
                                im6 = importances["importances_mean"][5]
                                im7 = importances["importances_mean"][6]
                                im8 = importances["importances_mean"][7]
                                im9 = importances["importances_mean"][8]
                                im10 = importances["importances_mean"][9]
                                sd1 = importances["importances_std"][0]
                                sd2 = importances["importances_std"][1]
                                sd3 = importances["importances_std"][2]
                                sd4 = importances["importances_std"][3]
                                sd5 = importances["importances_std"][4]
                                sd6 = importances["importances_std"][5]
                                sd7 = importances["importances_std"][6]
                                sd8 = importances["importances_std"][7]
                                sd9 = importances["importances_std"][8]
                                sd10 = importances["importances_std"][9]
                                tem = np.array([lr, c, d, s, t, est, itrain, jtrain, ival, jval, traincvtrain, f1s, mccs, rec, f1s2, mccs2, rec2, rs, im1, im2, im3, im4, im5, im6, im7, im8, im9, im10, sd1, sd2, sd3, sd4, sd5, sd6, sd7, sd8, sd9, sd10])
                                z = np.row_stack([z, tem])
                                #print(z[-1])
                            print("End of ", itr, " iterations")
                            itr += 1
    z_df = pd.DataFrame(z, columns=["lr", "cols", "depth", "subs", "tree", "est", 
                                    "f1_train", "mcc_train", "f1_ext", "mcc_ext", 
                                    "f1_5Ftrain", "f1_fna", "mcc_fna", "recall_fna", 
                                    "f1_di", "mcc_di", "recall_di", "state", 
                                    "im1", "im2", "im3", "im4", "im5", "im6", "im7", "im8", "im9", "im10", 
                                    "sd1", "sd2", "sd3", "sd4", "sd5", "sd6", "sd7", "sd8", "sd9", "sd10"])

    return z_df


In [19]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
## call Gradient Boost ##
optiSearch_df = optimXGBoostClass(trainDEFSDf, ingestedDEFSDf, extDEFSDf, fnaDEFSDf, diDEFSDf)

In [22]:
## save ##
optiSearch_df.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/raMSIn/modeling/hyperparameterTuning_modelSelection_XGB3.csv")

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [20]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [21]:
df_ROI_for_ML_Opti = pd.concat([trainDEFSDf, extDEFSDf, fnaDEFSDf, diDEFSDf]).set_index('pixel_id')

In [22]:
df_ROI_for_ML_Opti_ingested = ingestedDEFSDf.set_index('pixel_id')

In [23]:
df_ROI_for_ML_Opti_train = trainDEFSDf.set_index('pixel_id')

In [24]:
df_ROI_for_ML_Opti_ext = extDEFSDf.set_index('pixel_id')

In [25]:
df_ROI_for_ML_Opti_FNA = fnaDEFSDf.set_index('pixel_id')

In [26]:
df_ROI_for_ML_Opti_di = diDEFSDf.set_index('pixel_id')

In [27]:
df_ROI_for_ML_Opti_c = cDEFSDf.set_index('pixel_id')

In [28]:
df_ROI_for_ML_Opti_di.describe()

,311.1684,269.2486,215.0328,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,514.2846,type
count,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3028.000000
mean,3.556527e-17,4.993254e-16,4.882341e-16,3.667256e-16,-1.561205e-16,1.358080e-16,-3.366280e-16,1.970600e-15,1.163872e-14,5.951499e-16,0.500330
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.500082
min,-2.776735e+00,-3.684381e+00,-2.720374e+00,-3.328833e+00,-2.324191e+00,-1.701767e+00,-2.694045e+00,-1.475923e+00,-4.371089e-01,-2.716400e+00,0.000000
25%,-7.685900e-01,-6.134711e-01,-5.996852e-01,-5.392095e-01,-4.527378e-01,-6.450113e-01,-6.052840e-01,-5.787979e-01,-4.371089e-01,-7.348058e-01,0.000000
50%,9.775581e-02,-2.030662e-01,1.619175e-02,-2.568702e-02,8.349740e-02,-3.576545e-01,-1.598415e-01,1.457927e-01,-4.371089e-01,-4.128838e-01,1.000000
75%,7.284643e-01,3.619300e-01,6.212090e-01,5.052305e-01,5.960008e-01,2.576578e-01,3.912102e-01,7.469444e-01,1.578378e-03,7.484676e-01,1.000000
max,5.686665e+00,4.275386e+00,4.628362e+00,4.617973e+00,4.067668e+00,3.719943e+00,6.905744e+00,3.235670e+00,4.629420e+00,5.968980e+00,1.000000


In [29]:
df_ROI_for_ML_Opti_c.describe()

,311.1684,269.2486,215.0324,295.2278,883.5331,309.1704,738.5059,435.2965,250.1449,448.3072,type
count,6.711000e+03,6.711000e+03,6.711000e+03,6.711000e+03,6.711000e+03,6.711000e+03,6.711000e+03,6.711000e+03,6.711000e+03,6.711000e+03,6711.000000
mean,1.408334e-16,-2.969197e-16,2.212125e-15,4.867048e-17,7.186424e-16,1.098919e-14,-1.709423e-16,-1.704077e-14,1.066648e-15,4.358175e-15,0.486068
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.499843
min,-8.990175e-01,-1.939215e+00,-4.839536e-01,-6.727641e-01,-1.247431e+00,-5.619014e-01,-1.461826e+00,-4.344876e-01,-2.144975e+00,-6.004206e-01,0.000000
25%,-4.752203e-01,-7.912554e-01,-4.839536e-01,-6.727641e-01,-8.551104e-01,-5.619014e-01,-7.767353e-01,-4.344876e-01,-9.627713e-01,-6.004206e-01,0.000000
50%,-2.234467e-01,-2.409420e-01,-4.839536e-01,-6.727641e-01,-7.522237e-02,-5.619014e-01,-2.406599e-02,-4.344876e-01,1.337781e-01,-6.004206e-01,0.000000
75%,1.547694e-01,5.485696e-01,1.952751e-01,6.029558e-01,6.313744e-01,7.741586e-01,7.475702e-01,-4.344876e-01,8.445683e-01,5.935541e-01,1.000000
max,8.867916e+00,4.001617e+00,1.184330e+01,4.009780e+00,3.587549e+00,1.327993e+01,3.860615e+00,5.220208e+00,2.617374e+00,5.113009e+00,1.000000


## 2.2. PCA Analysis

In [20]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_c):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_c.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1', size=20)
    plt.ylabel('PC2', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/PCA_c.tif", bbox_inches = 'tight')

    components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()

pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [30]:
def split_dataset(df=df_ROI_for_ML_Opti_ingested):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2,random_state=42)
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti)

In [31]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]]

In [32]:
y_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[-1]]

In [33]:
X_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[:-1]]

In [34]:
y_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[-1]]

In [35]:
X_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[:-1]]

In [36]:
y_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[-1]]

In [37]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [38]:
y_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[-1]]

In [39]:
X_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[:-1]]

In [40]:
y_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[-1]]

In [41]:
X_c = df_ROI_for_ML_Opti_c[df_ROI_for_ML_Opti_c.columns[:-1]]

In [42]:
y_c = df_ROI_for_ML_Opti_c[df_ROI_for_ML_Opti_c.columns[-1]]

## 3.2 Machine Learning Modeling: Model 1+2 (Train 10:10, Val 4:4, Test 3:3)

In [43]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### 3.2.1. Logistic Regression

In [79]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di,  XTest3=X_c, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di, yTest3=y_c):
    model_lr = LogisticRegression(C=1, penalty='l1', solver="saga", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Set Precision:", precision_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_lr.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_lr.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_lr.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_lr.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_lr.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test3 set
    predictions_c = model_lr.predict(XTest3)
    print("Clincal Set F1:", f1_score(yTest3, model_lr.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set MCC:", matthews_corrcoef(yTest3, model_lr.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set Recall:", recall_score(yTest3, model_lr.predict(XTest3)))
    print("Clincal Set Precision:", precision_score(yTest3, model_lr.predict(XTest3)))
    print("Clincal Test Set Accuracy:", metrics.accuracy_score(yTest3, predictions_c))
    print(classification_report(yTest3, predictions_c))
    print(confusion_matrix(yTest3, predictions_c))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_c, tpr_c, _ = metrics.roc_curve(yTest3, predictions_c)
    plt.plot(fpr_c,tpr_c)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    return x, y

In [80]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.9396629596905028
Training Set MCC: 0.8791771441187178
Training Set Accuracy: 0.9395338610378188
              precision    recall  f1-score   support

           0       0.95      0.94      0.94     47449
           1       0.93      0.94      0.94     43511

    accuracy                           0.94     90960
   macro avg       0.94      0.94      0.94     90960
weighted avg       0.94      0.94      0.94     90960

[[44524  2925]
 [ 2575 40936]]
Ext Val Set F1: 0.753470095565255
Ext Val Set MCC: 0.49502474689656756
Ext Val Set Accuracy: 0.7479835390946502
              precision    recall  f1-score   support

           0       0.75      0.72      0.74      2943
           1       0.75      0.77      0.76      3132

    accuracy                           0.75      6075
   macro avg       0.75      0.75      0.75      6075
weighted avg       0.75      0.75      0.75      6075

[[2124  819]
 [ 712 2420]]
5F-CV: 0.906683561436146
FNA Set F1: 0.7528472668286127
FNA S

### 3.2.2. SVM

In [81]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di,  XTest3=X_c, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di, yTest3=y_c):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test3 set
    predictions_c = model_svm.predict(XTest3)
    print("Clincal Set F1:", f1_score(yTest3, model_svm.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set MCC:", matthews_corrcoef(yTest3, model_svm.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set Recall:", recall_score(yTest3, model_svm.predict(XTest3)))
    print("Clincal Set Precision:", precision_score(yTest3, model_svm.predict(XTest3)))
    print("Clincal Test Set Accuracy:", metrics.accuracy_score(yTest3, predictions_c))
    print(classification_report(yTest3, predictions_c))
    print(confusion_matrix(yTest3, predictions_c))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_c, tpr_c, _ = metrics.roc_curve(yTest3, predictions_c)
    plt.plot(fpr_c,tpr_c)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    return x, y

In [82]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.8720762725157191
Training Set MCC: 0.7345318454624576
Training Set Accuracy: 0.8521657871591909
              precision    recall  f1-score   support

           0       0.97      0.74      0.84     47449
           1       0.78      0.97      0.86     43511

    accuracy                           0.85     90960
   macro avg       0.87      0.86      0.85     90960
weighted avg       0.88      0.85      0.85     90960

[[35157 12292]
 [ 1155 42356]]
Ext Val Set F1: 0.7730969824145573
Ext Val Set MCC: 0.4981982696984242
Ext Val Set Accuracy: 0.7386008230452675
              precision    recall  f1-score   support

           0       0.85      0.56      0.67      2943
           1       0.69      0.91      0.78      3132

    accuracy                           0.74      6075
   macro avg       0.77      0.73      0.73      6075
weighted avg       0.77      0.74      0.73      6075

[[1639 1304]
 [ 284 2848]]
5F-CV: 0.8675377559826947
FNA Set F1: 0.8040494826216448
FNA 

### 3.2.3. GBM

In [83]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di,  XTest3=X_c, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di, yTest3=y_c):
    model_gbm = GradientBoostingClassifier(learning_rate=8, n_estimators=50, max_depth=6, min_samples_leaf=2, min_samples_split=50, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Set Precision:", precision_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    # Test2 set
    predictions_di = model_gbm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_gbm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_gbm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_gbm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_gbm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test3 set
    predictions_c = model_gbm.predict(XTest3)
    print("Clincal Set F1:", f1_score(yTest3, model_gbm.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set MCC:", matthews_corrcoef(yTest3, model_gbm.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set Recall:", recall_score(yTest3, model_gbm.predict(XTest3)))
    print("Clincal Set Precision:", precision_score(yTest3, model_gbm.predict(XTest3)))
    print("Clincal Test Set Accuracy:", metrics.accuracy_score(yTest3, predictions_c))
    print(classification_report(yTest3, predictions_c))
    print(confusion_matrix(yTest3, predictions_c))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_c, tpr_c, _ = metrics.roc_curve(yTest3, predictions_c)
    plt.plot(fpr_c,tpr_c)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    return x, y

In [84]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.7661586449924376
Training Set MCC: 0.49411208917180327
Training Set Accuracy: 0.7369393139841689
              precision    recall  f1-score   support

           0       0.82      0.64      0.72     47449
           1       0.68      0.85      0.75     43511

    accuracy                           0.74     90960
   macro avg       0.75      0.74      0.74     90960
weighted avg       0.75      0.74      0.73     90960

[[30182 17267]
 [ 6661 36850]]
Ext Val Set F1: 0.7096922356446335
Ext Val Set MCC: 0.3456759585069541
Ext Val Set Accuracy: 0.6689711934156378
              precision    recall  f1-score   support

           0       0.73      0.51      0.60      2943
           1       0.64      0.82      0.72      3132

    accuracy                           0.67      6075
   macro avg       0.68      0.66      0.66      6075
weighted avg       0.68      0.67      0.66      6075

[[1492 1451]
 [ 560 2572]]
5F-CV: 0.7445177895888574
FNA Set F1: 0.769989170858778
FNA 

### 3.2.4. Voting

In [63]:
# get a voting ensemble of models
def get_voting():
    models = list()
    # Define the base models
    models.append(("m1", LogisticRegression(C=1, penalty='l1', solver="saga", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    models.append(("m2", LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    models.append(("m3", GradientBoostingClassifier(learning_rate=8, n_estimators=50, max_depth=6, min_samples_leaf=2, min_samples_split=50, n_iter_no_change=5, random_state=42)))

    # define the voting ensemble
    ensemble = VotingClassifier(estimators=models, voting='hard')
    return ensemble

# get a list of models to evaluate
def get_models():
    models = dict()
    models['m1'] = LogisticRegression(C=1, penalty='l1', solver="saga", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    models['m2'] = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    models['m3'] = GradientBoostingClassifier(learning_rate=8, n_estimators=50, max_depth=6, min_samples_leaf=2, min_samples_split=50, n_iter_no_change=5, random_state=42)
    models['hard_voting'] = get_voting()
    return models


# evaluate a give model using cross-validation
def evaluate_model(model, XX, yy):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
    scores = cross_val_score(model, XX, yy, scoring='f1', cv=cv, n_jobs=-1, error_score='raise')
    return scores


In [64]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [65]:
# get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, XX=X_ingested, yy=y_ingested)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
    
# plot model performance for comparison
plt.subplots(dpi = 300)
plt.boxplot(results, labels=names, showmeans=True)
plt.ylabel('F1 score')
plt.xlabel('Predictive Models')
plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/raMSIn/modeling/voting_3models.jpg", bbox_inches = 'tight')

>m1 0.938 (0.001)
>m2 0.911 (0.015)
>m3 0.598 (0.195)
>hard_voting 0.931 (0.009)


In [68]:
# get a voting ensemble of models
def get_voting():
    models = list()
    # Define the base models
    models.append(("m1", LogisticRegression(C=1, penalty='l1', solver="saga", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    models.append(("m2", LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    #models.append(("m3", GradientBoostingClassifier(learning_rate=8, n_estimators=50, max_depth=6, min_samples_leaf=2, min_samples_split=50, n_iter_no_change=5, random_state=42)))

    # define the voting ensemble
    ensemble = VotingClassifier(estimators=models, voting='hard')
    return ensemble

# get a list of models to evaluate
def get_models():
    models = dict()
    models['m1'] = LogisticRegression(C=1, penalty='l1', solver="saga", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    models['m2'] = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    #models['m3'] = GradientBoostingClassifier(learning_rate=8, n_estimators=50, max_depth=6, min_samples_leaf=2, min_samples_split=50, n_iter_no_change=5, random_state=42)
    models['hard_voting'] = get_voting()
    return models


# evaluate a give model using cross-validation
def evaluate_model(model, XX, yy):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
    scores = cross_val_score(model, XX, yy, scoring='f1', cv=cv, n_jobs=-1, error_score='raise')
    return scores


In [69]:
# get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, XX=X_ingested, yy=y_ingested)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
    
# plot model performance for comparison
plt.subplots(dpi = 300)
plt.boxplot(results, labels=names, showmeans=True)
plt.ylabel('F1 score')
plt.xlabel('Predictive Models')
plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/raMSIn/modeling/voting_2models.jpg", bbox_inches = 'tight')

>m1 0.938 (0.001)
>m2 0.911 (0.015)
>hard_voting 0.924 (0.010)


In [75]:
def Voting_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di,  XTest3=X_c, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di, yTest3=y_c):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(X_train)
    print("Training Set F1:", f1_score(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Set Precision:", precision_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test3 set
    predictions_c = ensemble.predict(XTest3)
    print("Clincal Set F1:", f1_score(yTest3, ensemble.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set MCC:", matthews_corrcoef(yTest3, ensemble.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set Recall:", recall_score(yTest3, ensemble.predict(XTest3)))
    print("Clincal Set Precision:", precision_score(yTest3, ensemble.predict(XTest3)))
    print("Clincal Test Set Accuracy:", metrics.accuracy_score(yTest3, predictions_c))
    print(classification_report(yTest3, predictions_c))
    print(confusion_matrix(yTest3, predictions_c))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_c, tpr_c, _ = metrics.roc_curve(yTest3, predictions_c)
    plt.plot(fpr_c,tpr_c)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [76]:
Voting_func()

Training Set F1: 0.9385506919471346
Training Set MCC: 0.8775248442561534
Training Set Accuracy: 0.938896218117854
              precision    recall  f1-score   support

           0       0.94      0.94      0.94     47449
           1       0.94      0.94      0.94     43511

    accuracy                           0.94     90960
   macro avg       0.94      0.94      0.94     90960
weighted avg       0.94      0.94      0.94     90960

[[44700  2749]
 [ 2809 40702]]
Ext Val Set F1: 0.774368822410388
Ext Val Set MCC: 0.5557494154909429
Ext Val Set Accuracy: 0.777283950617284
              precision    recall  f1-score   support

           0       0.76      0.79      0.78      2943
           1       0.80      0.76      0.78      3132

    accuracy                           0.78      6075
   macro avg       0.78      0.78      0.78      6075
weighted avg       0.78      0.78      0.78      6075

[[2333  610]
 [ 743 2389]]
5F-CV: 0.8913812679982108
FNA Set F1: 0.7856901338025051
FNA Set

In [85]:
import joblib as jl

In [88]:
model = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
model.fit(X_ingested, y_ingested)

LinearSVC(C=402, class_weight={0: 0.9628, 1: 1.0402}, loss='hinge',
          random_state=42)

In [89]:
modelSavePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\modelSVM_raMSIn.joblib"
jl.dump(model, modelSavePath, compress = 5)

['I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\modeling\\modelSVM_raMSIn.joblib']

## 3.3 Machine Learning Modeling: Model 1 (Train 6:6, Val 4:4, Test 3:3)

### SVM

In [90]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di,  XTest3=X_c, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di, yTest3=y_c):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test3 set
    predictions_c = model_svm.predict(XTest3)
    print("Clincal Set F1:", f1_score(yTest3, model_svm.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set MCC:", matthews_corrcoef(yTest3, model_svm.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set Recall:", recall_score(yTest3, model_svm.predict(XTest3)))
    print("Clincal Set Precision:", precision_score(yTest3, model_svm.predict(XTest3)))
    print("Clincal Test Set Accuracy:", metrics.accuracy_score(yTest3, predictions_c))
    print(classification_report(yTest3, predictions_c))
    print(confusion_matrix(yTest3, predictions_c))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_c, tpr_c, _ = metrics.roc_curve(yTest3, predictions_c)
    plt.plot(fpr_c,tpr_c)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    return x, y

In [91]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.9129390274564335
Training Set MCC: 0.8325165648624774
Training Set Accuracy: 0.9168095866314864
              precision    recall  f1-score   support

           0       0.90      0.95      0.92     47449
           1       0.94      0.89      0.91     43511

    accuracy                           0.92     90960
   macro avg       0.92      0.92      0.92     90960
weighted avg       0.92      0.92      0.92     90960

[[44850  2599]
 [ 4968 38543]]
Ext Val Set F1: 0.4782266533546327
Ext Val Set MCC: 0.17003471308472554
Ext Val Set Accuracy: 0.5725102880658436
              precision    recall  f1-score   support

           0       0.54      0.77      0.64      2943
           1       0.64      0.39      0.48      3132

    accuracy                           0.57      6075
   macro avg       0.59      0.58      0.56      6075
weighted avg       0.59      0.57      0.56      6075

[[2268  675]
 [1922 1210]]
5F-CV: 0.8823191488675658
FNA Set F1: 0.7626737847149881
FNA

## 3.4 Machine Learning Modeling: Model 2 (Train 4:4, Val 6:6, Test 3:3)

### SVM

In [92]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di,  XTest3=X_c, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di, yTest3=y_c):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test3 set
    predictions_c = model_svm.predict(XTest3)
    print("Clincal Set F1:", f1_score(yTest3, model_svm.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set MCC:", matthews_corrcoef(yTest3, model_svm.predict(XTest3), sample_weight=sampleCW))
    print("Clincal Set Recall:", recall_score(yTest3, model_svm.predict(XTest3)))
    print("Clincal Set Precision:", precision_score(yTest3, model_svm.predict(XTest3)))
    print("Clincal Test Set Accuracy:", metrics.accuracy_score(yTest3, predictions_c))
    print(classification_report(yTest3, predictions_c))
    print(confusion_matrix(yTest3, predictions_c))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_c, tpr_c, _ = metrics.roc_curve(yTest3, predictions_c)
    plt.plot(fpr_c,tpr_c)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    return x, y

In [93]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.6995960215295853
Training Set MCC: 0.31712951713527826
Training Set Accuracy: 0.6422603342128408
              precision    recall  f1-score   support

           0       0.74      0.48      0.58     47449
           1       0.59      0.82      0.69     43511

    accuracy                           0.64     90960
   macro avg       0.67      0.65      0.64     90960
weighted avg       0.67      0.64      0.63     90960

[[22903 24546]
 [ 7994 35517]]
Ext Val Set F1: 0.9472325985078194
Ext Val Set MCC: 0.8971995948972225
Ext Val Set Accuracy: 0.9476543209876543
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      2943
           1       0.97      0.93      0.95      3132

    accuracy                           0.95      6075
   macro avg       0.95      0.95      0.95      6075
weighted avg       0.95      0.95      0.95      6075

[[2848   95]
 [ 223 2909]]
5F-CV: 0.8722363361665044
FNA Set F1: 0.6161725233792653
FNA

# 4. Model Evaluation

## 4.1 Prediction

In [94]:
## deploy models for training set ##
model = jl.load(r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\modelSVM_raMSIn.joblib")
#
## deploy model ##
predicted0n1_train = model.predict(X_train)      
trainDEFSDf["predicted0n1"] = predicted0n1_train
T_ = []
F_ = []
for i in range(len(trainDEFSDf)):
    if trainDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif trainDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
trainDEFSDf["T"] = T_
trainDEFSDf["F"] = F_
#
## save ##, ouputing trainSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_train_raMSIn_STDnorm_0n1.csv"
trainDEFSDf.to_csv(savePath)

In [95]:

## deploy model ##
predicted0n1_ext = model.predict(X_val)
extDEFSDf["predicted0n1"] = predicted0n1_ext
T_ = []
F_ = []
for i in range(len(extDEFSDf)):
    if extDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif extDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
extDEFSDf["T"] = T_
extDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_ext_raMSIn_STDnorm_0n1.csv"
extDEFSDf.to_csv(savePath)

In [96]:

## deploy model ##
predicted0n1_FNA = model.predict(X_FNA)
fnaDEFSDf["predicted0n1"] = predicted0n1_FNA
T_ = []
F_ = []
for i in range(len(fnaDEFSDf)):
    if fnaDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif fnaDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
fnaDEFSDf["T"] = T_
fnaDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_FNA_raMSIn_STDnorm_0n1.csv"
fnaDEFSDf.to_csv(savePath)

In [97]:

## deploy model ##
predicted0n1_Di = model.predict(X_di)
diDEFSDf["predicted0n1"] = predicted0n1_Di
T_ = []
F_ = []
for i in range(len(diDEFSDf)):
    if diDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif diDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
diDEFSDf["T"] = T_
diDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_Di_raMSIn_STDnorm_0n1.csv"
diDEFSDf.to_csv(savePath)

In [98]:

## deploy model ##
predicted0n1_C = model.predict(X_c)
cDEFSDf["predicted0n1"] = predicted0n1_C
T_ = []
F_ = []
for i in range(len(cDEFSDf)):
    if cDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif cDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
cDEFSDf["T"] = T_
cDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_Clinics3_raMSIn_STDnorm_0n1.csv"
cDEFSDf.to_csv(savePath)